In [ ]:
import time
import csv

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [ ]:

def open_front_page():
    driver.get("https://repertoire.sacem.fr/en/results?filters=parties&query=Eugene%20BOZZA#searchBtn", )

def accept_cookies():
    xpath_code = "//button[contains(text(), 'Accept Cookies')]"
    link = driver.find_element_by_xpath(xpath_code)
    link.click()
    
def click_show_following():
    xpath_code = "//button[contains(text(), 'show the following works')]"
    try:
        link = driver.find_element_by_xpath(xpath_code)
    except NoSuchElementException:
        return
    link.click()

def find_all_see_details():
    xpath_code = "//a[contains(text(), 'See detail')]"
    links = driver.find_elements_by_xpath(xpath_code)
    return links

In [ ]:
def do_an_item(link):
    driver.execute_script(f"window.open('{link}', 'new_window')")
    window_before = driver.window_handles[0]
    window_after = driver.window_handles[1]
    driver.switch_to.window(window_after)
    
    time.sleep(3)
    author_block = driver.find_elements_by_xpath('//h2[@data-reactid=".5.1.0"]')
    if len(author_block) != 1:
        print(f"not one author block -- danger {link}")
    author = author_block[0].text
    iswc_block = driver.find_elements_by_xpath('//p[@data-reactid=".5.1.2"]')
    if len(iswc_block) != 1:
        print(f"not one iswc block -- danger {link}")
    iswc = iswc_block[0].text
    driver.close()
    driver.switch_to.window(window_before)

    return author, iswc

In [ ]:
def do_all_items(all_detail_links):
    all_data = []
    for link_elem in all_detail_links:
        link = link_elem.get_attribute('href')
        title, iswc = do_an_item(link)
        all_data.append({'title': title, 'iswc': iswc})
    return all_data

In [ ]:
driver = webdriver.Firefox()

open_front_page()
time.sleep(5)
accept_cookies()
time.sleep(3)
for _ in range(20):
    click_show_following()
    time.sleep(1)

all_detail_links = find_all_see_details()
all_data = do_all_items(all_detail_links)

In [ ]:
all_data[0]

In [ ]:
import csv

In [ ]:
with open('/home/raleigh/Desktop/output.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=",",
                           quotechar='"', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['title', 'iswc'])
    for pair in all_data:
        spamwriter.writerow([pair['title'], pair['iswc']])